# Credit Banking Project

## 1.Load the Data

In [1]:
import pandas as pd

file_path = '/content/Credit Banking_Project - 1.xls'
customer = pd.read_excel(file_path, sheet_name=0)
spend = pd.read_excel(file_path, sheet_name=1)
repay = pd.read_excel(file_path, sheet_name=2)

# we load the dataset from xls file where we have 3 different sheets so we extract all sheet from file

## 2.Data Cleaning

In [2]:
customer['Limit'] = customer['Limit'].replace('[^0-9.]', '', regex=True).astype(float)

# Here we convert the data of limit column in float form. Ex: convert (INR 10,000.00) to (10000.00)
# [^0-9.] means rather than numbers from 0-9 or . will be remove we passed '' in second part that what value to replace

In [3]:
customer['Age'] = customer['Age'].apply(lambda x: 18 if x < 18 else x)

# we reframe the column for age below 18 because of data accuracy for credit card we replace below 18 Age with 18.

In [4]:
spend['Month'] = pd.to_datetime(spend['Month'])
repay['Month'] = pd.to_datetime(repay['Month'])

# we modify or also say that we change given month date in proper time format using to_datetime() function.

## 3.Task Analysis

### 3.1 Monthly Spend of Each Customer

In [5]:
per_month_spend = spend.groupby([spend['Costomer'], spend['Month'].dt.to_period('M')])['Amount'].sum().reset_index()
per_month_spend.columns = ['Customer', 'Month', 'Monthly_Spend']

per_month_spend.describe()
per_month_spend
# cus_month=monthly_spend.groupby(monthly_spend['Customer'])
# cus_month
#

,Customer,Month,Monthly_Spend
0,A1,2004-01,1.511173e+06
1,A1,2004-02,4.138111e+04
2,A1,2004-05,1.311966e+05
3,A1,2005-01,3.984038e+05
4,A1,2005-02,1.404193e+06
...,...,...,...
802,A95,2004-01,3.478339e+05
803,A96,2004-01,3.203635e+05
804,A97,2004-01,1.643300e+05
805,A98,2004-01,8.748351e+04


### 3.2 Monthly Repayment of Each Customer

In [6]:
per_month_repay = repay.groupby([repay['Costomer'], repay['Month'].dt.to_period('M')])['Amount'].sum().reset_index()
per_month_repay.columns = ['Customer', 'Month', 'Monthly_Repayment']

per_month_repay.describe()
per_month_repay

,Customer,Month,Monthly_Repayment
0,A1,2004-01,1.362775e+06
1,A1,2004-02,1.911800e+05
2,A1,2005-01,1.581970e+03
3,A1,2005-02,1.199808e+06
4,A1,2005-07,3.005817e+05
...,...,...,...
793,A95,2004-01,7.510949e+04
794,A96,2004-01,1.101390e+05
795,A97,2004-01,1.746064e+05
796,A98,2004-01,9.780260e+04



### 3.3 Top 10 Paying and Spending Customers

In [7]:
customers_spend_top_10 = spend.groupby('Costomer')['Amount'].sum().nlargest(10).reset_index()
customers_spend_top_10.columns=['Customer','Amount_Spend']
customers_spend_top_10

customers_repay_top_10 = repay.groupby('Costomer')['Amount'].sum().nlargest(10).reset_index()
customers_repay_top_10.columns=['Customer','Amount_Repay']
customers_repay_top_10

,Customer,Amount_Repay
0,A22,9.767171e+06
1,A60,9.262032e+06
2,A61,8.807888e+06
3,A40,8.805085e+06
4,A47,8.529826e+06
5,A43,8.458621e+06
6,A48,8.432804e+06
7,A41,8.374046e+06
8,A49,8.259841e+06
9,A45,8.115210e+06


In [12]:
spend.rename(columns={'Costomer': 'Customer'}, inplace=True)

### 3.4 Segment-wise Spending

In [13]:
merged_spend = pd.merge(spend, customer[['Customer', 'Segment']], on='Customer')
segment_spend = merged_spend.groupby('Segment')['Amount'].sum().reset_index()



segment_spend

,Segment,Amount
0,Govt,6.732563e+07
1,Normal Salary,1.077071e+08
2,Salaried_MNC,6.363949e+07
3,Salaried_Pvt,7.170431e+07
4,Self Employed,7.097548e+07


### 3.5 Age Group-wise Spending

In [16]:
def age_group(age):
    if age < 30: return '18-30'
    elif age < 45: return '30-45'
    elif age < 60: return '45-60'
    else: return '60+'

customer['Age_Group'] = customer['Age'].apply(age_group)
# customer
spend_age = pd.merge(spend, customer[['Customer', 'Age_Group']], on='Customer')
age_spend = spend_age.groupby('Age_Group')['Amount'].sum().reset_index()

age_spend


,Age_Group,Amount
0,18-30,1.395221e+08
1,30-45,4.448938e+07
2,45-60,6.324542e+07
3,60+,1.340952e+08


### 3.6 Category-wise Spending

In [18]:
category_spend = spend.groupby('Type')['Amount'].sum().sort_values(ascending=False).reset_index()
category_spend

,Type,Amount
0,PETRO,5.102258e+07
1,CAMERA,4.372101e+07
2,FOOD,3.829646e+07
3,AIR TICKET,3.743547e+07
4,TRAIN TICKET,3.181262e+07
5,SHOPPING,2.741868e+07
6,JEWELLERY,2.524795e+07
7,BUS TICKET,2.490590e+07
8,CLOTHES,2.479110e+07
9,RENTAL,2.091467e+07


### 3.7 Interest on Due Amount (2.9%)

In [30]:
# Merge repayment and spend
monthly = pd.merge(per_month_spend, per_month_repay, on=['Customer', 'Month'], how='left')

# monthly
monthly['Monthly_Repayment'] = monthly['Monthly_Repayment'].fillna(0)
# monthly

monthly['Due'] = monthly['Monthly_Spend'] - monthly['Monthly_Repayment']
monthly

monthly['Interest'] = monthly['Due'].apply(lambda x: x * 0.029 if x > 0 else 0)
monthly

monthly[['Interest','Customer','Month']]

,Interest,Customer,Month
0,4303.527824,A1,2004-01
1,0.000000,A1,2004-02
2,3804.701047,A1,2004-05
3,11507.832258,A1,2005-01
4,5927.177104,A1,2005-02
...,...,...,...
802,7909.007232,A95,2004-01
803,6096.511125,A96,2004-01
804,0.000000,A97,2004-01
805,0.000000,A98,2004-01


### 3.8 Monthly Profit for the Bank

In [32]:
monthly_profit = monthly.groupby('Month')['Interest'].sum().reset_index()
monthly_profit.head()

,Month,Interest
0,2004-01,577723.308202
1,2004-02,179315.885641
2,2004-03,91546.513730
3,2004-04,139859.166924
4,2004-05,82054.695624
